In [7]:
# Install required libraries

import os
from dotenv import load_dotenv
import logging # Import logging early

load_dotenv()

# Your familiar email sending/fetching variables
APP_PASSWORD = os.getenv('app_password')
SENDER_EMAIL = os.getenv('sender')

# Split recipients by comma if it's a string from .env
RECIPIENTS_STR = os.getenv('recipients1', '')
RECIPIENTS = [r.strip() for r in RECIPIENTS_STR.split(',') if r.strip()]

# For IMAP, you explicitly mentioned GMAIL_EMAIL in your example
GMAIL_IMAP_EMAIL = os.getenv("GMAIL_EMAIL", SENDER_EMAIL) # Fallback to SENDER_EMAIL if not set

# OpenRouter API Key for AI models
# OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")



In [8]:
OPENROUTER_API_KEY= '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'

In [9]:
def get_logger(name):
    logger= logging.getLogger(name) ## for calling get_logger("my app") my app will be logger name 
    logger.setLevel(logging.DEBUG)
    
    ch= logging.StreamHandler() ## direct all the output to (temrinal) console 
    ch.setLevel(logging.DEBUG)
    
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    
    if not logger.handlers:
        logger.addHandler(ch)

    return logger


In [10]:
# test 
logger= get_logger(__name__)
logger.info("Logger Testing Logger")

2025-07-24 23:21:19,705 - __main__ - INFO - Logger Testing Logger


## fetch emails

In [11]:
fetched_emails = []
num_emails_to_fetch = 5 # Let's fetch a small batch for testing purposes
IMAP_SERVER = "imap.gmail.com" 

In [12]:
import imaplib
mail = imaplib.IMAP4_SSL(IMAP_SERVER)
print(mail)
print()

## loging 
print() 

mail.login(user=GMAIL_IMAP_EMAIL , password=APP_PASSWORD )

if mail:
    print("login sucessfull")




login sucessfull


In [13]:
## continue fetch emails

mail.select('inbox') ## select the inbox folder 
status , messages= mail.search(None , 'ALL')

email_ids = messages[0].split() ## total email ids

print("status : " , status)# select inbox folder ,



print(f"Total messages you have  are {len(messages[0].split())}")



## get latest n emails UIDS 
latest_email_ids= email_ids[-num_emails_to_fetch:]



import email
## for uids 
for uid in latest_email_ids:
    status, msg_data= mail.fetch(uid, "(RFC822)") ## fetch raw email in the standard RFC 822 text format
    
    raw_email_bytes= msg_data[0][1]
    



status :  OK
Total messages you have  are 3089


In [14]:
raw_email_bytes[:100] ## the msg is prety messey it contains all the from to  , subject data stuff

b'Delivered-To: ultopalto0@gmail.com\r\nReceived: by 2002:a05:622a:438c:b0:4ab:8221:d0f1 with SMTP id em'

In [16]:
msg= email.message_from_bytes(raw_email_bytes)

## decoding inline headers 
subject_raw= msg.get('Subject' , 'No Subject') ## sometimes email is sent without writting subject so no subject 

print(f"SO the subject of this information was: ** {subject_raw} **")

from email.header import decode_header
decoded_parts= decode_header(subject_raw)
decoded_parts


print()
sender_raw = msg.get('From', 'Unknown Sender')
print("sender" , sender_raw)

SO the subject of this information was: ** Inquiry About Job Opportuniti **

sender Ujjwal Basnet <ujjwalvictus15@gmail.com>


In [17]:
msg.is_multipart

<bound method Message.is_multipart of <email.message.Message object at 0x000002943BB2B510>>

In [18]:
# check if msg is simple text or complex multi part
print(msg.is_multipart())

True


### now solving complex multipart like might contains pdf, links , embedded images , pain text version of msg , html style text  (solving body part)

In [19]:
for part in msg.walk(): 
    if part.get_content_type() == 'text/plain':
        body= part.get_payload(decode= True).decode()
        break

In [20]:
print(body)

Dear [Hiring Manager’s Name] / [Team neapl ]


I hope this message finds you well.


My name is Ujjwal Sharma, and I am currently exploring new career
opportunities where I can contribute meaningfully while continuing to grow
professionally. I am particularly interested in roles that align with my
skills in [insert field here – e.g., software development, data analysis,
content strategy], and your organization stood out to me as a place where
innovation and impact go hand in hand.


I would be grateful if you could let me know if there are any current or
upcoming openings that match my background. I have attached my résumé for
your reference, and I’d be happy to provide any additional details or
references if needed.


Thank you for your time and consideration. I look forward to the
possibility of connecting with you further.


Warm regards,

Ujjwal Sharma

[1245363]

[ujjwaexample@gmail.com



## Fetched_emails

In [21]:
fetched_emails.append({
                    "id": uid.decode(), # Store UID as string
                    "subject": subject_raw,
                    "sender": sender_raw,
                    "body": body,
                    "raw_message": raw_email_bytes # Keep raw message for debugging if needed
                })


In [22]:
fetched_emails[0].keys()

dict_keys(['id', 'subject', 'sender', 'body', 'raw_message'])

Successfully fetched

### now close and log out  of the server 

In [23]:
if 'mail' in locals() and mail.state == 'SELECTED':
                mail.close()
if 'mail' in locals() and mail.state == 'AUTH':
                mail.logout()


In [24]:
mail.state == "LOGOUT"

True

### select email 

In [50]:
class EmailState(TypedDict):
    """
    Represents the state of an email as it moves through the processing pipeline.
    """
    emails: List[Dict[str, Any]]
    current_email: Dict[str, Any]
    history: List[Dict[str, Any]]
    metadata: Dict[str, Any]


In [95]:
selected_email = None 

selected_email = fetched_emails 


if selected_email:
    current_email_state: EmailState = {
        'emails': [selected_email], # Keep it in a list to match the TypedDict structure
        'current_email': selected_email,
        'history': [], # To log steps taken
        'metadata': {} # To store classifications, etc.
    }

In [96]:
current_email_state['current_email'][0]['body'][:40] ## only priting 40 words

'Dear [Hiring Manager’s Name] / [Team nea'

In [97]:
## but   initially we set histroty and meta data  empty
print(current_email_state['history'])
print()

print(print(current_email_state['metadata'])
)


[]

{}
None


## LLM Model Initialization

In [98]:
model_filtering = None
model_summarization = None
model_response = None


In [99]:
from langchain_openai import ChatOpenAI

model_filtering = ChatOpenAI(
        base_url="https://openrouter.ai/api/v1",
        model="deepseek/deepseek-chat", # Using the specified model
        temperature=0.0,
        openai_api_key=OPENROUTER_API_KEY,
        max_retries=5
    )

In [100]:
#testing 
model_filtering.invoke("hi")

AIMessage(content='Hello!  😊 How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 4, 'total_tokens': 16, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 2}}, 'model_name': 'deepseek/deepseek-chat', 'system_fingerprint': '', 'id': 'gen-1753381729-FrCFlCt99ZFBv8oW8WnM', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--2e30a62d-6ef7-4d2b-9953-2a26810321fc-0', usage_metadata={'input_tokens': 4, 'output_tokens': 12, 'total_tokens': 16, 'input_token_details': {'cache_read': 2}, 'output_token_details': {}})

In [101]:
# Model for Summarization (moderate temperature for concise summaries)
model_summarization = ChatOpenAI(
        base_url="https://openrouter.ai/api/v1",
        model="deepseek/deepseek-chat",
        temperature=0.7,
        openai_api_key=OPENROUTER_API_KEY,
        max_retries=5
    )

if model_summarization:
    print("ok")

ok


In [102]:
# Model for Response Generation (higher temperature for more creative responses)

model_response = ChatOpenAI(
        base_url="https://openrouter.ai/api/v1",
        model="deepseek/deepseek-chat",
        temperature=0.8,
        openai_api_key=OPENROUTER_API_KEY,
        max_retries=5
    )

if model_response:
    print("ok")


ok


# Filter  Email

In [103]:
## now that we have selected email and also our llm is ready , lets test the filtering capability 
## this is classification task ,so low temperature is preferred for consistent result 

In [160]:
subject= current_email_state['current_email'][0].get('subject' , "")
print("subject: " , subject_to_filter) 

print() 

body = current_email_state['current_email'][0].get("body", "")
print("body : " ,  body [:50]) ## we are priting all body but here  only priting 50 words because this is hell lots of words , 


subject:  Inquiry About Job Opportuniti

body :  Dear [Hiring Manager’s Name] / [Team neapl ]





define prompt template for classfication 

In [161]:
# Langchain and LangGraph imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import END, StateGraph



filter_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert email classifier. Your task is to categorize emails into one of the following:
        - 'spam': Unsolicited commercial email, phishing attempts, or suspicious content.
        - 'urgent': Emails requiring immediate attention or action.
        - 'needs_review': Emails that require human discretion, contain ambiguous information, or might be important but not urgent.
        - 'informational': Emails that are for general information, newsletters, or routine updates, requiring no immediate action.

        Analyze the subject and body of the email to make an accurate classification.
        Output only the category name (e.g., 'spam', 'urgent', 'needs_review', 'informational')."""),
        ("human", "Subject: {subject}\n\nBody: {body}")
    ])


filter_chain = filter_prompt | model_filtering | StrOutputParser()



In [163]:
## get classification 
classification = filter_chain.invoke({"subject": subject, "body": body}).strip().lower()
classification


'needs_review'

In [165]:
valid_categories = {"spam", "urgent", "needs_review", "informational"}
if classification not in valid_categories:
    classification = 'needs_review'
    
 # Update the current_email_state with the classification

current_email_state['current_email'][0]["classification"] = classification # ouput -> 'needs_review'
current_email_state['metadata'][current_email_state['current_email'][0]['id']] = classification ## output -> {'3089': 'needs_review'}

current_email_state['history'].append({"step": "filter", "classification": classification}) ## output ->[{'step': 'filter', 'classification': 'needs_review'}]





if spam 

In [166]:
if classification == "spam":
    # Set a flag to skip subsequent steps for SPAM
    current_email_state['current_email'][0]['skip_ai_processing'] = True
    
else : 
     current_email_state['current_email'][0]['skip_ai_processing'] = False

# Summarize Email

In [176]:
if current_email_state['current_email'][0].get("skip_ai_processing") == False : 
    ## only run this if email is not spam , because if it is spam we dont need to proces anything just reject the email


    # Define the prompt template for summarization
    summary_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a concise email summarizer. Your task is to provide a brief,
            accurate summary of the email's key points. Focus on the main message and any
            actionable items. Keep the summary to 2-3 sentences."""),
            ("human", "Subject: {subject}\n\nBody: {body}")
        ])

    # Create the Langchain chain for summarization
    summary_chain = summary_prompt | model_summarization | StrOutputParser()


    # Invoke the chain to get the summary
    summary = summary_chain.invoke({"subject": subject, "body": body}).strip()
    current_email_state['current_email'][0]["summary"] = summary
    current_email_state['history'].append({"step": "summarize", "summary_len": len(summary)})

    print(summary)

Ujjwal Sharma is inquiring about job opportunities in a specific field (e.g., software development, data analysis) at the recipient's organization. He has attached his résumé and is open to providing further details or references if needed. Sharma expresses interest in roles that align with his skills and looks forward to potential discussions.


### Generate Response 

In [177]:
subject_for_response = current_email_state['current_email'][0].get("subject", "")
sender_for_response = current_email_state['current_email'][0].get("sender", "The sender")
summary_for_response = current_email_state['current_email'][0].get("summary", "")


In [180]:
subject_for_response , sender_for_response  , summary_for_response

('Inquiry About Job Opportuniti',
 'Ujjwal Basnet <ujjwalvictus15@gmail.com>',
 "Ujjwal Sharma is inquiring about job opportunities in a specific field (e.g., software development, data analysis) at the recipient's organization. He has attached his résumé and is open to providing further details or references if needed. Sharma expresses interest in roles that align with his skills and looks forward to potential discussions.")

In [183]:
sender_name = input("For the response signature, please enter your name  ")
reciver_name= input("for email greetign like (dear name) , please enter name")



response_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an AI email assistant. Your task is to craft a polite, relevant, and concise email reply.
        Consider the email's subject, sender, and the provided summary.

        If the email is 'informational', a brief acknowledgment or "thank you" is usually sufficient.
        If it's 'urgent' or 'needs_review', suggest next steps or ask clarifying questions.

        Aim for a professional and helpful tone. Do not include subject lines or greetings/signatures in your direct response,
        just the body of the reply."""),
        ("human", "Original Subject: {subject}\nFrom: {sender}\nSummary of email: {summary}\n\nGenerate a suitable reply:")
    ])


In [ ]:
response_chain = response_prompt | model_response | StrOutputParser()

In [226]:
response_content_raw = response_chain.invoke({
            "subject": subject,
            "sender": sender_raw,
            "summary": summary_for_response
        }).strip()


In [228]:
print(response_content_raw)

Thank you for reaching out and for your interest in joining our organization. We appreciate you sharing your résumé—we’ll review it carefully and keep it on file for any roles that align with your skills and experience.  

If there’s a potential match, we’ll be in touch to discuss next steps. In the meantime, feel free to let us know if there’s any additional information you’d like us to consider.  

We wish you the best in your job search and appreciate your time.


## send email 

In [229]:
response_content_filter= response_content_raw

In [230]:
your_name= "ujjwal" 


In [231]:
response_content_filter= response_content_filter.replace("[Your Name] " , your_name)
response_content_filter = response_content_filter.replace("[Your Position]", '')
response_content_filter = response_content_filter.replace("[Company Name]", '')

print(response_content_filter)

Thank you for reaching out and for your interest in joining our organization. We appreciate you sharing your résumé—we’ll review it carefully and keep it on file for any roles that align with your skills and experience.  

If there’s a potential match, we’ll be in touch to discuss next steps. In the meantime, feel free to let us know if there’s any additional information you’d like us to consider.  

We wish you the best in your job search and appreciate your time.


In [237]:
body_filled=response_content_filter
from email.mime.text import MIMEText


In [ ]:
subject, body, sender, recipients, password

'ujjwal_ai'

In [241]:
# --- Extract subject from body ---
if response_content_filter.startswith("Subject:"):
    subject_line, body_content = response_content_filter.split("\n\n", 1)
    subject = subject_line.replace("Subject:", "").strip()
else:
    subject = "No Subject"
    body_content = body_filled

# --- Function to send email ---
def send_email(subject, body, sender, recipients, password):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
            smtp_server.login(sender, password)
            smtp_server.sendmail(sender, recipients, msg.as_string())
        print("✅ Message sent successfully!")
    except Exception as e:
        print(f"❌ Failed to send email. Error: {e}")
        print("Make sure your sender address, app password, and connection are valid.")

# --- Send the email ---
send_email(subject, body_content, sender= SENDER_EMAIL, recipients=RECIPIENTS_STR, password=APP_PASSWORD )


✅ Message sent successfully!
